###INSTALL DEPENDENCIES AND IMPORT LIBRARIES

In [ ]:
!pip install transformers
!pip install datasets
!pip install sacrebleu
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 24.7 MB/s eta 0:00:0

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel
torch.cuda.empty_cache()

In [ ]:
!git clone https://github.com/nelson-nsc/COMP0087-NLP-project.git

Cloning into 'ucl-nlp'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 181 (delta 24), reused 27 (delta 17), pack-reused 137
Receiving objects: 100% (181/181), 7.98 MiB | 13.55 MiB/s, done.
Resolving deltas: 100% (66/66), done.


PREPARE DATA


In [ ]:
## HQ DATA
train_df = pd.read_csv('/content/COMP0087-NLP-project/data/conala-hq-marian/hq_train.csv')
test_df = pd.read_csv('/content/COMP0087-NLP-project/data/conala-hq-marian/hq_test.csv')
val_df = pd.read_csv('/content/COMP0087-NLP-project/data/conala-hq-marian/hq_val.csv')

In [ ]:
## HQ + MINED DATA
# hq_train_df = pd.read_csv('/content/COMP0087-NLP-project/data/conala-hq-marian/hq_train.csv')
# mined_df = pd.read_csv('/content/COMP0087-NLP-project/data/mined_data.csv')
# mined_df.dropna(inplace=True)

# train_df = pd.concat([hq_train_df, mined_df])
# train_df = train_df.sample(frac=1)
# train_df.reset_index(drop=True, inplace=True)

# test_df = pd.read_csv('/content/COMP0087-NLP-project/data/conala-hq-marian/hq_test.csv')
# val_df = pd.read_csv('/content/COMP0087-NLP-project/data/conala-hq-marian/hq_val.csv')

In [ ]:
## HQ + AUGMENTED DATA
# hq_train_df = pd.read_csv('/content/COMP0087-NLP-project/data/conala-hq-marian/hq_train.csv')
# aug_train_df = pd.read_csv('/content/COMP0087-NLP-project/data/conala-hq-marian/hq_train_aug.csv')

# train_df = pd.concat([hq_train_df, aug_train_df])
# train_df = train_df.sample(frac=1)
# train_df.reset_index(drop=True, inplace=True)

# test_df = pd.read_csv('/content/COMP0087-NLP-project/data/conala-hq-marian/hq_test.csv')
# val_df = pd.read_csv('/content/COMP0087-NLP-project/data/conala-hq-marian/hq_val.csv')


In [ ]:
train_df = train_df['intent'] + ' <ZZZ> ' + train_df['snippet']
val_df = val_df['intent'] + ' <ZZZ> ' + val_df['snippet']

###TOKENIZER AND MODEL

In [ ]:
# get gpt2 tokineizer and add our special token
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',  bos_token='<|startoftext|>',  eos_token='<|endoftext|>', pad_token='<|pad|>')
tokenizer.add_tokens([['<ZZZ>']])

model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()

model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50260, 768)

###PREPARE DATA PART 2

In [ ]:
max_length = max([len(tokenizer.encode(conala)) for conala in train_df])

In [ ]:
class coNaLaDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for i in data:
            encodings_dict = tokenizer('<|startoftext|>' + i + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
train_dataset = coNaLaDataset(train_df, tokenizer, max_length=max_length)
val_dataset = coNaLaDataset(val_df, tokenizer, max_length=max_length)

In [ ]:
my_data_collator = lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}

#### TRAIN THE MODEL

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/GPT2-FineTuned_mix_Z_3',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    per_device_train_batch_size=2, 
    per_device_eval_batch_size=2,
    num_train_epochs=3, 
    gradient_accumulation_steps = 2,
    warmup_steps=10, 
    weight_decay=0.05, 
    report_to = 'none',
    save_total_limit = 2,
    load_best_model_at_end = True,
    )


trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    data_collator = my_data_collator
)

trainer.train()

trainer.save_model('/content/drive/MyDrive/fineTunedGPT2_hq_Z_mix_3')
trainer.save_state()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.118000,0.091485
2,0.104200,0.079084
3,0.093100,0.074648


### HAVE FUN WITH THE MODEL

In [ ]:
# define the tokenizer and load in the trained model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',  bos_token='<|startoftext|>',  eos_token='<|endoftext|>', pad_token='<|pad|>', padding_side='left')
tokenizer.add_tokens([['<ZZZ>']])

model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/fineTunedGPT2_hq_Z_mix_3').cuda()
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50260, 768)

In [ ]:
## define a method to generate code from NL
def generate_response(NL, model, tokenizer, temp, n_out):

  inputs = tokenizer('<|startoftext|> ' + NL + ' <ZZZ> ', return_tensors='pt')

  input_ids = inputs.input_ids.to("cuda")
  attention_mask = inputs.attention_mask.to("cuda")

  outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, do_sample=True, top_k=50, max_length=75, top_p=0.99,
                                 temperature=temp, num_return_sequences=n_out, pad_token_id=tokenizer.encode('<|pad|>')[0])
  
  return [[tokenizer.decode(out, skip_special_tokens=True)] for out in outputs]

In [ ]:
generate_response('how to get mean of numpy array?', model, tokenizer, 0.001, 5)

[[' how to get mean of numpy array? <ZZZ>   y = np.mean(x, axis=0)'],
 [' how to get mean of numpy array? <ZZZ>   a = np.mean(a, axis=0)'],
 [' how to get mean of numpy array? <ZZZ>   y = np.mean(x, axis=0)'],
 [' how to get mean of numpy array? <ZZZ>   numpy.mean(a, axis=0)'],
 [' how to get mean of numpy array? <ZZZ>   numpy.mean(a, axis=0)']]